In [5]:
import torch
import numpy as np
import time
import timeit


In [32]:
n = 20 # Number of vectors
d = 20  # Dimensionality of vectors
v = torch.randn(n, d)
w = torch.rand(n)

# Method 0
def method_0(v, w):
    n, d = v.shape
    A = torch.zeros(d, d)
    for i in range(n):
        add = w[i] * torch.outer(v[i], v[i])
        A += add
    return A


# Method 1
def method_1(v, w):
    outerProds = torch.einsum('nd,nb->ndb', v, v)
    A = torch.einsum('ndb,n->db', outerProds, w)
    return A

# Method 2
def method_2(v, w):
    vScaled = torch.einsum('nd,n->nd', v, torch.sqrt(w))
    A = torch.einsum('nd,nb->db', vScaled, vScaled)
    return A

# Method 3
def method_3(v, w):
    Y = torch.einsum('nd,n->nd', v, w)
    A = torch.einsum('nd,nb->db', Y, v)
    return A

# Method 4 (normalized)
def method_4(v, w):
    n = w.size(0)
    Y = torch.einsum('nd,n->nd', v, w / n)
    A = torch.einsum('nd,nb->db', Y, v)
    return A

# Method 5 (direct einsum)
def method_5(v, w):
    A = torch.einsum('nd,n,nb->db', v, w, v)
    return A

sm0 = method_0(v, w)
sm1 = method_1(v, w)
sm2 = method_2(v, w)
sm3 = method_3(v, w)
sm4 = method_4(v, w) * n
sm5 = method_5(v, w)

# test if same result
epsilon = 1e-6
print(torch.allclose(sm0, sm1, atol=epsilon))
print(torch.allclose(sm0, sm2, atol=epsilon))
print(torch.allclose(sm0, sm3, atol=epsilon))
print(torch.allclose(sm0, sm4, atol=epsilon))
print(torch.allclose(sm0, sm5, atol=epsilon))
print(torch.allclose(sm2, sm3, atol=epsilon))

True
True
True
True
True
True


In [33]:
# Benchmark
n = 1000
d = 1000
v = torch.randn(n, d)
w = torch.rand(n)

time_0 = timeit.timeit('method_0(v, w)', globals=globals(), number=100)
time_1 = timeit.timeit('method_1(v, w)', globals=globals(), number=100)
time_2 = timeit.timeit('method_2(v, w)', globals=globals(), number=100)
time_3 = timeit.timeit('method_3(v, w)', globals=globals(), number=100)
time_4 = timeit.timeit('method_4(v, w)', globals=globals(), number=100)
time_5 = timeit.timeit('method_5(v, w)', globals=globals(), number=100)

print(f"Method 0: {time_0}")
print(f"Method 1: {time_1}")
print(f"Method 2: {time_2}")
print(f"Method 3: {time_3}")
print(f"Method 4: {time_4}")
print(f"Method 5: {time_5}")



Method 0: 32.39974782199715
Method 1: 92.3654925370065
Method 2: 0.316125652010669
Method 3: 0.3198895480018109
Method 4: 0.3250637330056634
Method 5: 0.32300357600615826


## Test pytorch vs numpy speed on CPU

In [15]:
# Create random matrices
matrix_size = 1000
np_matrix1 = np.random.rand(matrix_size, matrix_size)
np_matrix2 = np.random.rand(matrix_size, matrix_size)
torch_matrix1 = torch.tensor(np_matrix1)
torch_matrix2 = torch.tensor(np_matrix2)

# PyTorch matrix multiplication
start = time.time()
torch_result = torch.matmul(torch_matrix1, torch_matrix2)
end = time.time()
print(f"PyTorch time: {end - start} seconds")

# NumPy matrix multiplication
start = time.time()
np_result = np.matmul(np_matrix1, np_matrix2)
end = time.time()
print(f"NumPy time: {end - start} seconds")

PyTorch time: 0.01648545265197754 seconds
NumPy time: 0.016958236694335938 seconds


In [18]:
matrix_size = 1000
num_runs = 100

# Create random matrices using NumPy
np_matrix1 = np.random.rand(matrix_size, matrix_size)
np_matrix2 = np.random.rand(matrix_size, matrix_size)

# Convert NumPy matrices to PyTorch tensors
torch_matrix1 = torch.tensor(np_matrix1, dtype=torch.float32)
torch_matrix2 = torch.tensor(np_matrix2, dtype=torch.float32)

# Define functions for matrix multiplication
def numpy_multiplication():
    res = np.matmul(np_matrix1, np_matrix2)
    return res

def torch_multiplication():
    res = torch.matmul(torch_matrix1, torch_matrix2)
    return res

# Time NumPy matrix multiplication using timeit
numpy_time = timeit.timeit(numpy_multiplication, number=num_runs) / num_runs
print(f'Average time for NumPy matrix multiplication: {numpy_time:.6f} seconds')

# Time PyTorch matrix multiplication using timeit
torch_time = timeit.timeit(torch_multiplication, number=num_runs) / num_runs
print(f'Average time for PyTorch matrix multiplication: {torch_time:.6f} seconds')

Average time for NumPy matrix multiplication: 0.007726 seconds
Average time for PyTorch matrix multiplication: 0.004296 seconds
